# Knowledge Distillation For Fine-Tuning A GPT-3.5 Judge

There has been recent research that demonstrated GPT-4's ability to closely align to human judges when evaluating LLM generated texts (e.g., see [[1]](https://arxiv.org/abs/2306.05685), [[2]](https://arxiv.org/abs/2303.16634)). In this notebook, we demonstrate how to use the `llama_index` library to distill knowledge from GPT-4 to GPT-3.5 so that the smaller GPT-3.5 becomes closer to GPT-4 performance; and by proxy, closer to human judges.

To do so, we take the following steps:

1. Generate datasets: `train` and `test`
2. Perform knowledge distillation (using `train`)
3. Evaluate the distilled model  on `test`

More specifically, we will use `CorrectnessEvaluator` as our LLM Judge in this notebook.

```
IMO the easiest evaluator to fine-tune for is probably the correctness metric. generate "ground-truth dataset" of question + ground-truth context -> ground-truth answer, and then use GPT-4 as the correctness evaluator to compare predicted answer from RAG vs. ground-truth answer, and fine-tune to 3.5
```

## 0 Prompt Templates & Auxiliary Functions

In [ ]:
import os

HUGGING_FACE_TOKEN = os.getenv("HUGGING_FACE_TOKEN")

In [ ]:
PROMPTS = {
    "QUESTION_GEN": (
        "You are a Teacher/ Professor. Your task is to setup "
        "a quiz/examination. Using the provided context, formulate "
        "a single question that captures an important fact from the "
        "context. Restrict the question to the context information provided."
    )
}

In [ ]:
import pandas as pd


# define jupyter display function
def display_eval_df(question, source, answer_a, answer_b, result) -> None:
    eval_df = pd.DataFrame(
        {
            "Question": question,
            "Source": source,
            "Model A": answer_a["model"],
            "Answer A": answer_a["text"],
            "Model B": answer_b["model"],
            "Answer B": answer_b["text"],
            "Score": result.score,
            "Judgement": result.feedback,
        },
        index=[0],
    )
    eval_df = eval_df.style.set_properties(
        **{
            "inline-size": "300px",
            "overflow-wrap": "break-word",
        },
        subset=["Answer A", "Answer B"]
    )
    display(eval_df)

## 1 Generate datasets: `train` and `test`

As mentioned before the LLM Judge we will be using is the `CorrectnessEvaluator`. What this means then is that we'll need to generate some ground-truth data from which our Judge can assert whether or not the provided response is indeed correct.

So, the LLM Judge will get a `<query, generated-answer, ground-truth-answer>`, and it will output a correctness score between 0 and 1 (higher is better).

We will handle getting `<query, ground-truth-answer>` separately from `<generated-answer>`.

### Generate `<query, ground-truth-answer>`

We're going to turn a set of Wikipedia pages into `Documents` and define a `DatasetGenerator` on top of them to generate our `queries` and `ground-truth-answers`. 

In [ ]:
!pip install wikipedia -q


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
# wikipedia pages
from llama_index.readers import WikipediaReader

cities = [
    "San Francisco",
    "Toronto",
    "New York",
    "Vancouver",
    "Montreal",
]

documents = WikipediaReader().load_data(
    pages=[f"History of {x}" for x in cities]
)

In [ ]:
# generate questions against chunks
from llama_index.evaluation import DatasetGenerator
from llama_index.llms import OpenAI
from llama_index import ServiceContext

# set context for llm provider
gpt_35_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3)
)

# instantiate a DatasetGenerator
dataset_generator = DatasetGenerator.from_documents(
    documents,
    question_gen_query=PROMPTS["QUESTION_GEN"],
    service_context=gpt_35_context,
    show_progress=True,
    num_questions_per_chunk=25,
)

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
qrd = dataset_generator.generate_dataset_from_nodes(num=100)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.56s/it]


In [ ]:
# save for future
qrd.save_json("qrd.json")

### Create `generated-answers`

In [ ]:
from llama_index import VectorStoreIndex
from llama_index.indices.vector_store.retrievers import VectorIndexRetriever

# Create vector index
the_index = VectorStoreIndex.from_documents(documents=documents)

# Create the retriver on this index
the_retriever = VectorIndexRetriever(
    index=the_index,
    similarity_top_k=2,
)

In [ ]:
from llama_index.query_engine.retriever_query_engine import (
    RetrieverQueryEngine,
)
from llama_index.llms import HuggingFaceInferenceAPI
from llama_index.llm_predictor import LLMPredictor

llm = HuggingFaceInferenceAPI(
    model_name="meta-llama/Llama-2-7b-chat-hf",
    context_window=2048,  # to use refine
    token=HUGGING_FACE_TOKEN,
)
context = ServiceContext.from_defaults(llm_predictor=LLMPredictor(llm=llm))
query_engine = RetrieverQueryEngine.from_args(
    retriever=the_retriever, service_context=context
)

In [ ]:
import tqdm

dataset = []
num_train_questions = int(0.65 * len(qrd.qr_pairs))
for q, a in tqdm.tqdm(qrd.qr_pairs[:num_train_questions]):
    # data for this q
    data_entry = {"question": q, "reference": a}
    response = query_engine.query(q)
    response_struct = {}
    response_struct["model"] = "llama-2"
    response_struct["text"] = str(response)
    response_struct["context"] = (
        response.source_nodes[0].node.text[:1000] + "..."
    )

    data_entry["response_data"] = response_struct
    dataset.append(data_entry)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:13<00:00,  2.82it/s]


### Generate Answers From GPT-4 Judge

In [ ]:
# instantiate the gpt-4 judge
from llama_index.llms import OpenAI
from llama_index import ServiceContext
from llama_index.callbacks import OpenAIFineTuningHandler
from llama_index.callbacks import CallbackManager
from llama_index.evaluation import CorrectnessEvaluator

# NOTE: this finetuning_handler will collect 2x chat_histories for
# each query: one for original, and another for flipped
finetuning_handler = OpenAIFineTuningHandler()
callback_manager = CallbackManager([finetuning_handler])
gpt_4_context = ServiceContext.from_defaults(
    llm=OpenAI(temperature=0, model="gpt-4"),
    callback_manager=callback_manager,
)

gpt4_judge = CorrectnessEvaluator(service_context=gpt_4_context)

In [ ]:
import tqdm

# for `training`
for data_entry in tqdm.tqdm(dataset):
    eval_result = await gpt4_judge.aevaluate(
        query=data_entry["question"],
        response=data_entry["response_data"]["text"],
        context=data_entry["response_data"]["context"],
        reference=data_entry["reference"],
    )

    # save final result
    judgement = {}
    judgement["llm"] = "gpt_4"
    judgement["score"] = eval_result.score
    judgement["text"] = eval_result.response
    data_entry["evaluations"] = [judgement]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [04:11<00:00,  6.62s/it]


In [ ]:
finetuning_handler.save_finetuning_events("correction_finetuning_events.jsonl")

Wrote 38 examples to correction_finetuning_events.jsonl


## Fine-tuning GPT-3.5

In [ ]:
from llama_index.finetuning import OpenAIFinetuneEngine

finetune_engine = OpenAIFinetuneEngine(
    "gpt-3.5-turbo",
    "correction_finetuning_events.jsonl",
)

In [ ]:
finetune_engine.finetune()

Num examples: 38
First example:
{'role': 'system', 'content': '\nYou are an expert evaluation system for a question answering chatbot.\n\nYou are given the following information:\n- a user query,\n- a reference answer, and\n- a generated answer.\n\nYour job is to judge the relevance and correctness of the generated answer.\nOutput a single score that represents a holistic evaluation.\nYou must return your response in a line with only the score.\nDo not return answers in any other format.\nOn a separate line provide your reasoning for the score as well.\n\nFollow these guidelines for scoring:\n- Your score has to be between 1 and 5, where 1 is the worst and 5 is the best.\n- If the generated answer is not relevant to the user query, you should give a score of 1.\n- If the generated answer is relevant but contains mistakes, you should give a score between 2 and 3.\n- If the generated answer is relevant and fully correct, you should give a score between 4 and 5.\n\nExample Response:\n4.0\

In [ ]:
finetune_engine.get_current_job()

<FineTuningJob fine_tuning.job id=ftjob-mecbP0ruMxHEqOAsnF7eCydF at 0x2aa3ae660> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-mecbP0ruMxHEqOAsnF7eCydF",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1698790732,
  "finished_at": 1698791412,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:llamaindex::8FrXRqaU",
  "organization_id": "org-1ZDAvajC6v2ZtAP9hLEIsXRz",
  "result_files": [
    "file-dHUwbAvZK0NKi466LBMy6HB1"
  ],
  "status": "succeeded",
  "validation_file": null,
  "training_file": "file-ge2lmJw8Illb0tjagxG4LqCd",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 54156,
  "error": null
}

## LLM Judgements 

First let's prepare our `test` set.

In [ ]:
test_dataset = []
for q, a in tqdm.tqdm(qrd_pairs[num_train_questions:]):
    # data for this q
    data_entry = {"question": q, "reference": a}
    response = query_engine.query(q)
    response_struct = {}
    response_struct["model"] = "llama-2"
    response_struct["text"] = str(response)
    response_struct["context"] = (
        response.source_nodes[0].node.text[:1000] + "..."
    )

    data_entry["response_data"] = response_struct
    test_dataset.append(data_entry)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:06<00:00,  3.17it/s]


Now, let's predict on our test set.

In [ ]:
import tqdm

# for `test`
for data_entry in tqdm.tqdm(test_dataset):
    eval_result = await gpt4_judge.aevaluate(
        query=data_entry["question"],
        response=data_entry["response_data"]["text"],
        context=data_entry["response_data"]["context"],
        reference=data_entry["reference"],
    )

    # save final result
    judgement = {}
    judgement["llm"] = "gpt_4"
    judgement["score"] = eval_result.score
    judgement["text"] = eval_result.response
    data_entry["evaluations"] = [judgement]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [02:12<00:00,  6.31s/it]


In [ ]:
ft_llm = finetune_engine.get_finetuned_model()
ft_context = ServiceContext.from_defaults(
    llm=ft_llm,
)
ft_gpt_3p5_judge = CorrectnessEvaluator(service_context=ft_context)

# a non-fine-tuned judge
gpt_3p5_context = ServiceContext.from_defaults(
    llm=OpenAI(model="gpt-3.5-turbo")
)
gpt_3p5_judge = CorrectnessEvaluator(service_context=gpt_3p5_context)

In [ ]:
from llama_index.evaluation import EvaluationResult

# predicting on training set for now just to get rest of pipeline established
for data_entry in tqdm.tqdm(test_dataset):
    eval_result = await ft_gpt_3p5_judge.aevaluate(
        query=data_entry["question"],
        response=data_entry["response_data"]["text"],
        context=data_entry["response_data"]["context"],
        reference=data_entry["reference"],
    )

    # save final result
    judgement = {}
    judgement["llm"] = "ft_gpt_3p5"
    judgement["score"] = eval_result.score
    judgement["text"] = eval_result.response
    data_entry["evaluations"] += [judgement]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:20<00:00,  1.03it/s]


In [ ]:
# predicting on training set for now just to get rest of pipeline established
for data_entry in tqdm.tqdm(test_dataset):
    eval_result = await gpt_3p5_judge.aevaluate(
        query=data_entry["question"],
        response=data_entry["response_data"]["text"],
        context=data_entry["response_data"]["context"],
        reference=data_entry["reference"],
    )

    # save final result
    judgement = {}
    judgement["llm"] = "gpt_3p5"
    judgement["score"] = eval_result.score
    judgement["text"] = eval_result.response
    data_entry["evaluations"] += [judgement]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 21/21 [00:41<00:00,  1.97s/it]


## Evaluation the LLM Judges

In [ ]:
REPORT_FMT_STR = (
    "{model}\n"
    "-----------------\n"
    "Number of obs.: {total_obs}\n"
    "Correlation with GPT-4: {corr}\n"
)

In [ ]:
import numpy as np

scores = {"gpt_4": [], "gpt_3p5": [], "ft_gpt_3p5": []}
for ix, d in enumerate(test_dataset):
    for e in d["evaluations"]:
        scores[e["llm"]].append(e["score"])

In [ ]:
# numpy conversion
np_scores_gpt_4 = np.array(scores["gpt_4"])
np_scores_gpt_3p5 = np.array(scores["gpt_3p5"])
np_scores_ft_gpt_3p5 = np.array(scores["ft_gpt_3p5"])

# correlations
corr_ft = np.corrcoef(np_scores_gpt_4, np_scores_ft_gpt_3p5)[0, 1]
corr_no_ft = np.corrcoef(np_scores_gpt_4, np_scores_gpt_3p5)[0, 1]

print(
    REPORT_FMT_STR.format(
        model="GPT-3.5 w/ fine-tuning",
        total_obs=np_scores_gpt_4.shape[0],
        corr=corr_ft,
    )
)
print("\n")
print(
    REPORT_FMT_STR.format(
        model="GPT-3.5 w/out fine-tuning",
        total_obs=np_scores_gpt_4.shape[0],
        corr=corr_no_ft,
    )
)

GPT-3.5 w/ fine-tuning
-----------------
Number of obs.: 21
Correlation with GPT-4: 0.6901319161879755



GPT-3.5 w/out fine-tuning
-----------------
Number of obs.: 21
Correlation with GPT-4: 0.7103381407680267

